<a href="https://colab.research.google.com/github/tryfasting/deeplearning-pytorch-textbook/blob/main/11-02%20Bag%20of%20Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 단어의 등장 순서를 고려하지 않는 빈도수 기반 단어 표현 방법인 Bag of Words에 대해서 배운다.

### 1. Bag of Words란

단어들의 순서는 고려하지 않고, 단어들의 frequency에만 집중한다.    

1. 각 단어에 고유한 정수 인덱스를 부여한다.  
2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.

**문서 1 : 정부가 발표하는 물가상승률과 소비자가 느기는 물가상승률은 다르다.**  

에 대해서 BoW를 만들어보자.

In [2]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 35.4 MB/s eta 0:00:00


In [7]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.','')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:
        if word not in word_to_index.keys():
            # 자동으로 새로운 단어에 value를 len(word_to_index)만큼 부여
            # 새로운 value는 유일한 값을 갖는 것을 보장받는다.
            # 따라서 인덱스는 중복되지 않고 부여된다.
            word_to_index[word] = len(word_to_index)
            # Bow에 전부 기본값 1를 넣는다.ㅠ
            bow.insert(len(word_to_index)-1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다
            bow[index] = bow[index] + 1

            # word_to_index 말 그대로, 단어에 인덱스를 부여해 딕셔너리로 만든다.
            # 즉 Bow의 해당 인덱스에 접근하면, 단어의 frequency를 구할 수 있다.

    return word_to_index, bow


해당 함수에 문서1을 입력으로 넣어보자.

In [8]:
doc1 = '정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.'
vocab, bow = build_bag_of_words(doc1)
print('vocabulary : ', vocab)
print('bag of words vector : ',bow)

vocabulary :  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector :  [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


만약 한국어에서 불용어에 해당하는 조사들을 제거한다면 더 정제된 BoW를 만들 수도 있겠다.

### 2. Bag of Words의 다른 예제들

**문서 2 : 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.**  

위의 함수에 임의의 문서 2를 입력하여 결과를 확인해보자.

In [9]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'
vocab, bow = build_bag_of_words(doc2)
print('vocabulary : ', vocab)
print('bag of words vector : ',bow)

vocabulary :  {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector :  [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


문서 1, 문서 2를 합쳐서 문서 3이라고 명명하고, BoW를 만들 수도 있다.  

**문서3 : 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.**

In [10]:
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


BoW는 종종 여러 문서의 단어 집합을 합친 뒤에, 해당 단어 집합에 대한 각 문서의 BoW를 구하기도 한다.  
가령, 문서3에 대한 단어 집합을 기준으로 문서1, 문서2의 BoW를 만들면 결과는 아래와 같다.

문서3 단어 집합에 대한 문서1 BoW : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]  
문서3 단어 집합에 대한 문서2 BoW : [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1]  


BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법이므로 문서가 어떤 성격의 문서인지를 판단하는 작업에 쓰인다.

### 3. CountVectorizer 클래스로 BoW 만들기

사이킷런의 CountVectorizer 클래스로 BoW를 만들어보자.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know i want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector : ', vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary : ', vector.vocabulary_)

bag of words vector :  [[1 1 2 1 2 1]]
vocabulary :  {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


기본적으로 CountVectorizer는 길이가 2 이상인 문자에서는 토큰으로 인식하기 때문에, 'I' 같은 문자는 빠졌다. 정제(Cleaning) 챕터에서 언급했듯이, 영어에서는 길이가 짧은 문자를 제거하는 것 또한 전처리 작업으로 고려되기도 한다.  

 CountVectorizer는 띄어쓰기를 기준으로 단어를 자르기 때문에, 한국어에는 적용하는 건 적절치 않다.

### 4. 불용어를 제거한 BoW 만들기

BoW를 만들어서, 각 단어에 대한 빈도수를 수치화하겠다는 것은, 즉 텍스트 내에서 어떤 단어가 중요한지 살펴보겠다는 뜻이다. 그렇다면 불용어를 제거해서 자연어처리의 정확도를 높이는 것이 좋겠다.

영어는 CountVectorizer에서, 불용어를 제외하고 BoW를 만들어주는 기능을 지원한다.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

#### 1) 사용자가 직접 정의한 불용어 사용

In [15]:
text = ['Family is not an important thing. It\'s everything.']
vect = CountVectorizer(stop_words=['the','a','an','is','not'])

print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :', vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


#### 2) CountVectorizer에서 제공하는 자체 불용어 사용

In [16]:
text = ['Family is not an important thing. It\'s everything.']
vect = CountVectorizer(stop_words='english')

print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :', vect.vocabulary_)

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


#### 3) NLTK에서 지원하는 불용어 사용

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
text = ['Family is not an important thing. It\'s everything.']
stop_words = stopwords.words('english')
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :', vect.vocabulary_)

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}
